# Data Science: Bridging Principle and Practice
## Part 7: Linear Regression Model (Bike Sharing case study)

<br/>

<div class="container">
    <div style="float:left;width:40%">
	    <img src="images/bikeshare_sun.jpg">
    </div>
    <div style="float:left;width:40%">
	    <img src="images/bikeshare_snow.PNG">
    </div>
</div>

### Table of Contents

<ol start="7">
    <li><a href="section 7">Linear Regression Model</a>
        <ol type=a>
            <br>
            <li><a href="section7a">Explanatory and Response Variables</a></li>
            <br>
            <li><a href="section7b">Finding &beta;</a></li>
            <br>
            <li><a href="section7c">Evaluating the Model</a></li>            
        </ol>
    </li>
    </ol>

In [ ]:
# run this cell to import some necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import seaborn as sns


import ipywidgets as widgets
from scipy.linalg import lstsq

def make_X(df, var_names):
    """Given a DataFrame and a list of explanatory variables, one-hot encodes
    variables if they are categorical and returns a dataframe with 
    all the given explanatory variables."""
    data = df.loc[:, var_names]
    categorical = ["month", "week day", "season"]
    boolean = ["is holiday", "is work day"]
    X = pd.DataFrame({"intercept":np.ones(df.shape[0], dtype='int')})
    for var in data.columns:
        if var in categorical:
            dummies = pd.get_dummies(data[var])
            formatted = dummies.drop(dummies.columns[-1], axis=1)
        elif var in boolean:
            formatted = (data[var] == "yes") * 1
        else:
            formatted = data.loc[:, [var]]
        X = X.join(formatted)
      
    return X

## Case Study: Capital Bike Share <a id= "sectioncase"></a>

Bike-sharing systems have become increasingly popular worldwide as environmentally-friendly solutions to traffic congestion, inadequate public transit, and the "last-mile" problem. Capital Bikeshare runs one such system in the Washington, D.C. metropolitan area.

The Capital Bikeshare system comprises docks of bikes, strategically placed across the area, that can be unlocked by *registered* users who have signed up for a monthly or yearly plan or by *casual* users who pay by the hour or day. They collect data on the number of casual and registered users per hour and per day.

Let's say that Capital Bikeshare is interested in a **prediction** problem: predicting how many riders they can expect to have on a given day. [UC Irvine's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) has combined the bike sharing data with information about weather conditions and holidays to try to answer this question.

In this notebook, we'll walk through the steps a data scientist would take to answer this question.

In [ ]:
# run this cell to load the data
bike_train = pd.read_csv("../resource/data/day_renamed_dso.csv", )

# load the test data
bike_test = pd.read_csv("../resource/data/day_test.csv")

# reformat the date column from strings to dates
bike_train['date'] = pd.to_datetime(bike_train['date'])
bike_test['date'] = pd.to_datetime(bike_test['date'])

## 7. The Regression Model <a id= "section7"></a>

To try to predict the number of riders on a given day, we'll use a regression model. 

> A **simple regression model** describes how the conditional mean of a response variable $y$ depends on an explanatory variable $x$: $$\mu_{y|x} = \beta_0 + \beta_1x$$ This equation describes our best guess for $y$ given a particular  $x$.

> A **multiple regression model** describes how the conditional mean of a response variable $y$ depends on multiple explanatory variables $x_1, x_2, ..., x_k$: $$\mu_{y|x_1, x_2, ..., x_k} = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k$$ This equation describes our best guess for $y$ given a particular vector $x_1, x_2, ..., x_k$.

In this case, our model will look something like this:
$$\mu_{\text{rider count}|\text{temp, humidity, ..., month}} = \beta_0 + \beta_1*\text{(temp)} + \beta_2*\text{(humidity)} + ... + \beta_k*\text{(month)}$$
The code for either a simple or multiple regression model is basically identical except for the number of columns we select for inclusion in our explanatory variables.

To create our model, we need to:
1. isolate the explanatory variables (X) and response variable (y) we want to use
2. find the values for the $\beta$ variables on the best-fit regression line
3. evaluate the accuracy of our model

### 7a. Explanatory and response variables <a id="subsection7a"></a>

First, let's decide on our response variable. We'll try to predict the *total number of riders* on a given day. The response variable needs to be in an array (not a DataFrame), so we'll get it using indexing.

In [ ]:
# response variable: total count of riders in a day (training set)
y_train = bike_train['total riders']

# response variable: total count of riders in a day (validation set)
y_test = bike_test['total riders']

# show the first 5 items in the array
y_train.head()

Next, we want to choose our explanatory variables. Let's try predicting ridership in terms of _temperature_, _work day_, and _season_.

<div class="alert alert-info">

<b>Why don't we just use all the explanatory variables?</b>
<p>
    You might think that the best model would use <i>all</i> the available explanatory information. But, using many variables makes a model <b>computationally expensive</b>. In the real world, where data sets may have a million or more rows, using a complex model can increase the time and computing power needed to make preditions. Additionally, many variables may have <b>little predictive power</b> such that excluding them from the model doesn't lower the accuracy very much. Other variables might <b>add noise</b> that actually decreases the model's performance outside of the training data.
    </p>

</div> 

Here, we run into a problem: our DataFrame has several categorical variables, including "season" and "week day". Categorical variables seem like they might be helpful in predicting the number of riders; however, we can't feed text into a mathematical model. And, if we code them as integers, as they were coded in the original data set (e.g. Monday = 1, Tuesday = 2, ...) that can lead to questionable manipulations. For example, if Sunday is coded as 0 and Saturday is coded as 6, the computer might conclude that the average of Sunday and Saturday is Wednesday (since Wednesday is coded as 3).

#### One-Hot Encoding
To work around this, we will **one-hot encode** all our categorical variables. In one-hot encoding, the possible values of the variable each get their own column of 1s and 0s. The value is a 1 if that day falls in that category and a 0 otherwise.

Here's an example. Say we have three possible weather states: rain, cloudy, or sunny.

In [ ]:
# an example DataFrame of weather states
categorical = pd.DataFrame({"weather": ["rainy", "cloudy", "sunny", "cloudy"]})
categorical

The one-hot encoding would look like this:

In [ ]:
# pd.get_dummies is a function that does dummy encoding
one_hot = pd.get_dummies(categorical['weather'])
one_hot

Notice that in each row, only one of the values is equal to 1 (hence the name, "one-hot" encoding), since no day can have more than one weather state.

Notice also that we don't technically need the third column. 

In [ ]:
one_hot.drop("sunny", axis=1)

If we know that there are only three possible weather states, and we see that day 2 was neither cloudy nor rainy (that is, `cloudy`=0 and `rainy`=0), day 2 *must* have been sunny. This is helpful to save computation time and space. If you have some knowledge of linear algebra, note that this is also helpful to solve the problem of *perfect multicollinearity*- a situation that can make it impossible to compute the optimal set of $\beta$s.

For simplicity, we've provided a function called `make_X` that will convert our data to the correct format for prediction, including one-hot encoding the categorical variables. `make_X` takes two arguments:
* the first argument, `df`, is the DataFrame with the data we want to use to make predictions
* the second argument, `var_names`, is a list with the names of all the explanatory variables in `df` we want our model to use to make predictions: categorical and numerical

`make_X` will also add a column called "intercept" that only contains 1s. This column will help find the intercept term $\beta_0$ in our regression line. You can think of the intercept term as an $x_0$ that gets multiplied by $\beta_0$ and is always equal to 1.

In [ ]:
# select the explanatory variables to use in a DataFrame
expl_vars = ["temp", "is work day", "season"]

# convert the explanatory table to the correct format
X_train = make_X(bike_train, expl_vars)

# show the first five rows of the model inputs
X_train.head()

<div class="alert alert-warning">
<b>EXERCISE:</b> 
    <p>
        Since we want to try the model on the test data as well, we will also perform the same transformations on the test set so it can fit the model. Fill in the code to first select the explanatory variables "temp", "is work day", and "season", then convert the explanatory table to the matrix format using <code>format_X</code>. 
    </p>
    <p>Hint: we'll need to go through the exact same steps as in the above cell for the training data, but any references to training data should be replaced by their test data counterparts.
    </p>
</div>

In [ ]:
# select the explanatory variables to use in a Table
expl_vars = ...

# convert the explanatory table to the correct format
X_test = ...

# show the first five rows of the model inputs
X_test.head()

### 7b.  Finding $\beta$ <a id="subsection7b"></a>
The next step is to calculate the $\beta$ terms. We can do this with a function from the [Scipy statistical analysis Python package](https://www.scipy.org/) called `lstsq`.

Given a matrix of explanatory variables X and an array of response variables y, `lstsq` returns a vector $\beta$. `lstsq` uses *ordinary least squares* as its **loss function (L)**: the function that defines the training loss (error) and what we seek to minimize (often using linear algebra or calculus, depending on the loss function). The ordinary least squares equation is a common loss function that is used to minimize the sum of squared errors:

$$L(\beta) = \frac{1}{n}\sum_{i=1}^{n}(y_i - \text{predicted}(\beta, x_i))^2$$

where $n$ is the number of days, $y_i$ is the actual number of riders on the $i$th day, and $\text{predicted}(\beta, x_i)$ is number of riders predicted to be on the $i$th day when using $\beta$ and the explanatory variables $x_i$. When minimized, the loss function will yield our optimal $\beta$. 

`lstsq` returns a list of four things, but for our purposes we're only interested in one of them: the array of the $\beta$ values for the best-fit line. If you're curious about exactly how `lstsq` works, you can [read the docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html) for more information, but that isn't necessary to complete this notebook.

In [ ]:
# calculate the least squares solution
lstsq_results = lstsq(X_train, y_train)

beta = lstsq_results[0]

beta

We now have everything we need to make predictions about the total number of riders on a given day. Remember, the formula is: $$\mu_{y|x_1, x_2, ..., x_k} = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k$$

So, to make a prediction for a given day, we take all the values in the X matrix corresponding to that day, multiply each value by its corresponding $\beta$, then add them all together. The `@` operator can help us with this matrix multiplication.

For example, here's the first row of our explanatory variable matrix.

In [ ]:
X_train.loc[0, :]

To get the prediction, we use `@` to multiply each item in the row by each corresponding item in the $\beta$ vector and sum them all up. If you've taken linear algebra, you'll recognize this as the [*dot product*](https://en.wikipedia.org/wiki/Dot_product).

<img src="images/vector_mult.png" />



In [ ]:
# multiply the arrays to get the prediction
X_train.loc[0, :] @ beta

The `@` operator can also work on matrices. To get the predictions for *every* row in X, we use exactly the same syntax.

<img src="images/matrix_mult.png"  />


In [ ]:
# take the dot product of every row of training data and beta
# the resulting array is a predicted number of total riders for each day
predict_train = X_train @ beta

# show the first 5 predictions
predict_train.head()

Now we can add our predictions to our original table.

In [ ]:
# make a new column in our training data with the predicted total riders
bike_train["predicted total riders"] = predict_train

# show the first five rows of the DataFrame
bike_train.head()

<div class="alert alert-warning">
    <b>EXERCISE:</b> We also want to make predictions for the test data using the $\beta$ we found during training. Replace the <code>...</code> in the cell below with an expression to calculate the predictions for the test set. Remember- you need to use <code>@</code> to multiply each row of explanatory variables in our test set by the $\beta$ vector. Look at how <code>predict_train</code> was calculated for a guide.
    </div>

In [ ]:
# make predictions for the test data using beta and the dot product
predict_test = ...

# create a new column in our test data with predicted total riders
bike_test["predicted total riders"] = predict_test

# show the first 5 rows of test data
bike_test.head()

### 7c. Evaluating the model <a id="subsection7c"></a>

Our model makes predictions, but how good are they? We can start to get a sense of how we did by plotting the predictions versus the actual values on our training data on a scatter plot. If our model predicts perfectly:

- the predicted values will be equal to the actual values
- all points in the scatter plot will fall along a straight line with a slope of 1

To do this, we'll use a Seaborn function called `regplot`. `regplot` creates the scatter plot, then adds the linear regression line of best fit. The translucent band around the line represents the 95% confidence interval for the regression estimate- we'll talk more about confidence intervals in a future notebook.

In [ ]:
# make the plots bigger
sns.set(rc={'figure.figsize':(11.7,8.27)})

# create the scatter plot and regression line
sns.regplot(x="total riders", y="predicted total riders", data=bike_train);


Here are the test set predictions scattered against the actual total rider counts. Note that we've added an extra `color` argument to `regplot` to change the color of the dots and line and distinguish the test data from the training data.

In [ ]:
# create the scatter plot and regression line
sns.regplot(x="total riders", y="predicted total riders", data=bike_test, color=(1.0, 200/256, 44/256));

### Root Mean Squared Error
Now it would be nice to have a quantitative measure of how good our model is. An intuitive way to evaluate our model is to look at the *error* for each prediction: the size of the difference between the number of riders we predicted and the actual number of riders. We can get this for each item in our data set by **subtracting the array of predictions from the array of actual values**:

In [ ]:
# get an array of errors
errors = y_train - predict_train

# show the first five items in the error array
errors.head()

Now we know the error for each individual prediction, but how is the model doing in general? Let's try taking the average of the errors using the `np.average` function:

In [ ]:
# take the average of the errors
np.average(errors)

The `e-13` is scientific notation: it means that we should take the number to the left of it and multiply it by $10^{-13}$ (0.0000000000001). The resulting number is very, very close to zero. So, if the average error is almost zero, that would seem to imply that our model is, on average, making near perfect predictions. We know this is untrue thanks to our scatter plot.

What happened?

The problem is that about half of our errors are negative (overestimation) and the other half are positive (underestimation). This is actually a good thing- it means that our model isn't systematically over or underestimating. However, it means that averaging the errors is not a good quantitative measurement of our model, since the negative errors cancel out the positive errors.

In this case, we care about the *magnitude* of the errors: how far each prediction is from the actual value regardless of whether the prediction is over or under. To make all of our errors positive while retaining the magnitude, let's **square all of our errors**. In Python, `**` is the exponential operation; `3 ** 2` tells the computer to multiply 3 by itself 2 times.

In [ ]:
# square the array of errors
sq_error = errors ** 2

# show the first five squared errors
sq_error.head()

Now that none of our errors are negative, we can safely **take the average**:

In [ ]:
# take the average of the squared errors
mean_sq_error = np.average(sq_error)
mean_sq_error

The units for the mean squared error are squared riders, which is hard to interpret in real life. Ideally, we would want an average error measure in *riders* so we could say that on average, our model is off by this many riders.

Since mean squared error is in squared riders, we can get a number with a unit of riders by **taking the square root** of our mean squared error:

In [ ]:
# take the square root of the mean squared errors
root_mean_sq_err = np.sqrt(mean_sq_error)
root_mean_sq_err

This number is the **Root Mean Squared Error** (RMSE, sometimes also called the Root Mean Squared Deviation). RMSE is our quantitative measure of the average performance of our model. To summarize the steps:

1. Find the difference between the actual and predicted value for each of our data points (the errors)
2. Square all of the errors
3. Take the average of the squared errors
4. Take the square root of the average squared error

If you like formulas, here it is in a formula:

$$rmse=\sqrt{\frac{\sum_{i=1}^n (y_i - prediction_i)^2}{n}}$$

Next, we want to see what the RMSE is for our test set predictions. 

<div class="alert alert-warning">
    <b>EXERCISE:</b> Calculate the root mean squared error for the test set. Follow each of the above steps, and look at how it was calculated for the training set for some hints.
</div>

Before you run the next cell: would you expect the RMSE for the test set would be higher, lower, or about the same as the RMSE for the training set? Why?

In [ ]:
# calculate the error for each data point
test_errors = ...

# square the errors
test_sq_error = ...

# take the mean of the squared errors
test_mean_sq_error = ...

# take the square root of the mean squared errors
test_rmse = ...
test_rmse

### Visualizing Error
We can also visualize our errors compared to the actual values on a scatter plot.

In [ ]:
# plot the training errors on a scatter plot
bike_train["training error"] = errors
sns.regplot(x="predicted total riders", y="training error", data=bike_train);

In [ ]:
# plot the test errors on a scatter plot
bike_test["test error"] = y_test - predict_test
sns.regplot(x="predicted total riders", y="validation error", data=bike_test, color=(1.0, 200/256, 44/256));

<div class="alert alert-warning">
    <b>QUESTION:</b> Based on the plots and root mean squared error above, how well do you think our model is doing? What does the shape of the scatter plot of errors tell us about the appropriateness of the linear model here?
    </div>

**ANSWER:** 

#### References
- Bike-Sharing data set from University of California Irvine's Machine Learning Repository https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
- Portions of text and code adapted from Professor Jonathan Marshall's Legal Studies 190 (Data, Prediction, and Law) course materials: [lab 2-22-18, Linear Regression](https://github.com/ds-modules/LEGALST-190/tree/master/labs/2-22) (Author Keeley Takimoto)  and [lab 3-22-18, Exploratory Data Analysis](https://github.com/ds-modules/LEGALST-190/tree/masterlabs/3-22) (Author Keeley Takimoto)
- "Capital Bikeshare, Washington, DC" header image by [Leeann Caferatta](https://www.flickr.com/photos/leeanncafferata/34309356871) licensed under [CC BY-ND 2.0](https://creativecommons.org/licenses/by-nd/2.0/)